## 1. Load Data

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import os
import numpy as np
import pandas as pd
import nltk
import gensim
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from PIL import Image
from wordcloud import WordCloud

import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

path = "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

In [59]:
movies_org = pd.read_csv(path + "movies.csv")
movies = pd.read_csv(path + "movies_crawling.csv")
ratings = pd.read_csv(path + "ratings.csv")
tags = pd.read_csv(path + "tags.csv")

In [60]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [61]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [62]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## 2. EDA

In [63]:
# Restore TS
time_unit = 3
ratings['timestamp'] = ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))
tags['timestamp'] = tags['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))

print(min(ratings['timestamp']), max(ratings['timestamp']))
print(min(tags['timestamp']), max(tags['timestamp']))

1978-09-30 22:12:18.333333 1986-03-30 20:49:10
1982-01-05 06:23:04 1986-03-28 03:56:41


In [36]:
# Find External N/A -> None
print(movies.isna().sum())
print()
print(ratings.isna().sum())
print()
print(tags.isna().sum())

movieId    0
title      0
genres     0
dtype: int64

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64


In [37]:
# Genres
lzGenre= []
for index, row in movies.iterrows():
  try:
    genres= row.genres.split("|")
    lzGenre.extend(genres)
  except: lzGenre.append(row.genres)

lzGenre= list(set(lzGenre))
print(lzGenre)
print(len(lzGenre))

['Indie film', 'Animation', 'Western', 'Silent', 'Sci-fi', 'Comedy', 'Documentary', 'Fantasy', 'Adventure', 'Action', 'Mystery', 'Short', 'Film-Noir', 'IMAX', 'Dance', 'Drama', 'Musical', 'Sport', 'Romance', 'War', 'Thriller', 'Horror', '(no genres listed)', 'Children', 'Crime', 'Sci-Fi']
26


## 3. Title Column Preproc


In [38]:
# Elimination of any blanks in title
for i in range(len(movies)):
  ttl= movies.iloc[i,1].strip()
  movies.iloc[i,1]= ttl

print(movies.iloc[5609,1])

From Dusk Till Dawn 2: Texas Blood Money (1999)


In [39]:
# Fix Title
for i in range(len(movies)):
  movieID= movies.iloc[i,0]
  title= movies.iloc[i,1]
  if title[-5:] == ', The': movies.iloc[i,1]= 'The ' + title[:-5]
  elif title[-4:] == ', An': movies.iloc[i,1]= 'An ' + title[:-4]
  elif title[-3:] == ', A': movies.iloc[i,1]= 'A ' + title[:-3]

## 3. Year Column, Tag Column, Era Column

In [40]:
# Add Year Column
lzYR= []
for i in range(len(movies)):
  yr= movies.iloc[i,1][-8:][3:-1]
  if yr.isnumeric():
    ttl= movies.iloc[i,1][:-7]
    movies.iloc[i,1]= ttl
    lzYR.append(int(yr))
  else: lzYR.append(np.NaN)

movies['year']= lzYR
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [41]:
# Genre Column : get rid of |
for i in range(len(movies)):
  genre= movies.iloc[i,2]
  genre= genre.replace('|', ' ')
  if genre=='(no genres listed)' : genre= np.NaN
  # else: genre= genre.split(' ')[0]
  movies.iloc[i,2]= genre

movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0
1,2,Jumanji,Adventure Children Fantasy,1995.0
2,3,Grumpier Old Men,Comedy Romance,1995.0
3,4,Waiting to Exhale,Comedy Drama Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


Silent Era (Late 19th century to late 1920s): This era marked the early days of cinema when films were made without synchronized sound. Silent movies relied on visual storytelling, often accompanied by live music. Notable directors like Charlie Chaplin and Buster Keaton emerged during this time.

Golden Age of Hollywood (1920s to 1960s): Hollywood's studio system was at its peak during this period. It saw the rise of glamorous studios like MGM, Paramount, and Warner Bros. Movies were characterized by their star-driven plots, genres like musicals, film noir, and the establishment of the Hays Code, a set of censorship guidelines.

New Hollywood (Late 1960s to 1980s): This era marked a departure from traditional studio practices. Filmmakers embraced more experimental and unconventional storytelling, often tackling controversial subjects. Directors like Martin Scorsese, Francis Ford Coppola, and Steven Spielberg gained prominence during this time.

Blockbuster Era (Late 1970s to present): The late 1970s brought a shift towards big-budget, high-concept films designed for mass appeal. Movies like "Jaws" and "Star Wars" set the stage for the blockbuster trend that continues to influence Hollywood today.

Contemporary Era (2000s to present): This period encompasses the digital revolution in filmmaking. Advances in technology have expanded the possibilities for storytelling and special effects. It's characterized by a mix of franchise blockbusters, independent films gaining wider recognition, and the rise of streaming services influencing distribution.

In [42]:
# Add TS Category Column
lzTSCat = []
for i in range(len(movies)):
    yr = movies.iloc[i, 3]
    if yr < 1930: lzTSCat.append("S")
    elif 1930 <= yr < 1970: lzTSCat.append("G")
    elif 1970 <= yr < 1990: lzTSCat.append("N")
    elif 1990 <= yr < 2000: lzTSCat.append("B")
    elif pd.isnull(yr): lzTSCat.append(np.NaN)
    else: lzTSCat.append("C")

movies['era'] = lzTSCat
movies.head()

,movieId,title,genres,year,era
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0,B
1,2,Jumanji,Adventure Children Fantasy,1995.0,B
2,3,Grumpier Old Men,Comedy Romance,1995.0,B
3,4,Waiting to Exhale,Comedy Drama Romance,1995.0,B
4,5,Father of the Bride Part II,Comedy,1995.0,B


In [46]:
movies.isna().sum()

movieId     0
title       0
genres     10
year       12
era        12
dtype: int64

## 4. Tag Preproc

In [43]:
lzMovieID= list(set(tags['movieId']))
lzTTL= []
lzTag= []
for id in lzMovieID:
  lzTTL.append(str(movies[movies['movieId']==id]['title'].values)[2:-2])
  dfTags= tags[tags['movieId']==id]
  lzTag.append(str(set(dfTags['tag']))[1:-1])

dfCombTag= pd.DataFrame()
dfCombTag['movieId']= lzMovieID
dfCombTag['title']= lzTTL
dfCombTag['tag']= lzTag
dfCombTag.head()

,movieId,title,tag
0,1,Toy Story,"'fun', 'pixar'"
1,2,Jumanji,"'fantasy', 'magic board game', 'game', 'Robin ..."
2,3,Grumpier Old Men,"'moldy', 'old'"
3,122882,Mad Max: Fury Road,"'cinematography', 'visually appealing', 'beaut..."
4,5,Father of the Bride Part II,"'remake', 'pregnancy'"


In [44]:
TagCorpus = dfCombTag.tag.values

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
def Clean(doc):
  tokens= word_tokenize(doc.lower())
  tokens= [word for word in tokens if word.isalpha() and not word in stop_words]
  return tokens

mv_tags_doc = [TaggedDocument(words=Clean(D), tags=[str(i)]) for i, D in enumerate(TagCorpus)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [45]:
epoch = 50
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size= vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=0)
model.build_vocab(mv_tags_doc)

print('Epoch', end = ': ')
for ep in range(epoch):
  model.train(mv_tags_doc,
              total_examples=model.corpus_count,
              epochs=model.epochs)
  model.alpha -= 0.0002
  model.min_alpha = model.alpha

Epoch: 

In [ ]:
TagVec = model.docvecs.vectors
TagVec.shape

<ipython-input-42-d6ad8eb349a2>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  TagVec = model.docvecs.vectors


(1572, 20)

In [ ]:
dfCombTag['tag_vector'] = list(TagVec)

In [ ]:
dfCombTag.head()

,movieId,title,tag,tag_vector
0,1,Toy Story,"'pixar', 'fun'","[-0.026154077, -0.029895633, -0.049403768, 0.0..."
1,2,Jumanji,"'game', 'fantasy', 'magic board game', 'Robin ...","[-0.20993754, 0.24306962, -0.2731193, -1.15787..."
2,3,Grumpier Old Men,"'moldy', 'old'","[-0.010284597, -0.03615021, 0.020923, -0.04287..."
3,122882,Mad Max: Fury Road,"'visually appealing', 'cinematography', 'beaut...","[-0.057963945, -0.4769408, -0.04419995, -0.623..."
4,5,Father of the Bride Part II,"'pregnancy', 'remake'","[0.04174178, 0.023339575, -0.047050517, -0.001..."


In [ ]:
movies.head()

,movieId,title,genres,year,era
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0,B
1,2,Jumanji,Adventure Children Fantasy,1995.0,B
2,3,Grumpier Old Men,Comedy Romance,1995.0,B
3,4,Waiting to Exhale,Comedy Drama Romance,1995.0,B
4,5,Father of the Bride Part II,Comedy,1995.0,B


In [ ]:
movies_fix= pd.merge(movies, dfCombTag, on= 'movieId')
movies_fix.head()

,movieId,title_x,genres,year,era,title_y,tag,tag_vector
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0,B,Toy Story,"'pixar', 'fun'","[-0.026154077, -0.029895633, -0.049403768, 0.0..."
1,2,Jumanji,Adventure Children Fantasy,1995.0,B,Jumanji,"'game', 'fantasy', 'magic board game', 'Robin ...","[-0.20993754, 0.24306962, -0.2731193, -1.15787..."
2,3,Grumpier Old Men,Comedy Romance,1995.0,B,Grumpier Old Men,"'moldy', 'old'","[-0.010284597, -0.03615021, 0.020923, -0.04287..."
3,5,Father of the Bride Part II,Comedy,1995.0,B,Father of the Bride Part II,"'pregnancy', 'remake'","[0.04174178, 0.023339575, -0.047050517, -0.001..."
4,7,Sabrina,Comedy Romance,1995.0,B,Sabrina,'remake',"[0.044488966, 0.03123408, -0.031191802, 0.0433..."


In [ ]:
movies_fix.to_csv('movies_fix.csv', index= False, encoding= 'utf-8')